# Example Evaluation Code

This notebook will be very __similar__ to the code I use to evaluate your results - it is provided for __your convenience__ so that you can use it to evaluate your preprocessing results at any time before your __final submission__.

Please note that the results here will __NOT__ be the same as my evaluation results.

Let's start with loading the required packages.

In [1]:
# import required package for data handling
import pandas as pd
import numpy as np

# import required packages for splitting data
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# import required packages for evaluating models
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

# import `logistic regression` model
from sklearn.linear_model import LogisticRegression

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import shapiro
import seaborn as sns
from scipy.stats import zscore, skew
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from itertools import chain
import xgboost
from sklearn import metrics, model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.decomposition import PCA

from sklearn.metrics import roc_curve, confusion_matrix

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', 500)

Next you should load __your__ data. In this case, I am using a sample dataset (`GroupX.csv`) which contains 6 predictors (`X1 - X6`) and two target variables (`Y1, Y2`).

Please make sure you change the data to your __OWN__ dataset when using this code.

__NOTE__:
1. Your dataset maybe very different from the sample dataset.
2. Please follow this structure when submitting your dataset.

In [3]:
feat_mat = pd.read_csv('feat_mat.csv', header=0).drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'])
tar = pd.read_csv('target_df.csv')

In [4]:
y2 = tar.Y2

In [58]:
high_corr_map = {'C7_imputed_iqr_standardized_normalized': ['sales_per_share_iqr_standardized_normalized'],
 'words_per_sentence_iqr_standardized_normalized': ['short_sentences_per_iqr_standardized_normalized',
  'long_sentences_per_iqr_standardized_normalized'],
 'real_word_per_iqr_standardized_normalized': ['non_real_word_per_iqr_standardized_normalized'],
 'non_real_word_per_iqr_standardized_normalized': ['real_word_per_iqr_standardized_normalized'],
 'short_sentences_per_iqr_standardized_normalized': ['words_per_sentence_iqr_standardized_normalized',
  'long_sentences_per_iqr_standardized_normalized'],
 'long_sentences_per_iqr_standardized_normalized': ['words_per_sentence_iqr_standardized_normalized',
  'short_sentences_per_iqr_standardized_normalized'],
 'neg_words_percent_iqr_standardized_normalized': ['uncertain_words_percent_iqr_standardized_normalized'],
 'uncertain_words_percent_iqr_standardized_normalized': ['neg_words_percent_iqr_standardized_normalized'],
 'sales_per_share_iqr_standardized_normalized': ['C7_imputed_iqr_standardized_normalized'],
 'outstanding_share_per_iqr_standardized_normalized': ['offering_share_per_iqr_standardized_normalized',
  'C5_prime_iqr_standardized_normalized'],
 'offering_share_per_iqr_standardized_normalized': ['outstanding_share_per_iqr_standardized_normalized',
  'C5_prime_iqr_standardized_normalized'],
 'C5_prime_iqr_standardized_normalized': ['outstanding_share_per_iqr_standardized_normalized',
  'offering_share_per_iqr_standardized_normalized']}

In [5]:
def get_important_features_rfe(xTrain, yTrain, num_feats):
    '''
    Takes training values and returns the most important features in order
    '''
    clf = LogisticRegression(random_state=123, max_iter = 1000)
    selector = RFE(clf, num_feats, step=1)
    selector.fit(xTrain, yTrain)
    
    return [x for _,x in sorted(zip(selector.ranking_, list(xTrain.columns)))]
    
    
feat_rfe = get_important_features_rfe(feat_mat[[i for i in list(feat_mat.columns) 
                                     if i != 'I1'and 'iqr_standardized_normalized' in i]],
                          y2, 10)

In [65]:
for i in high_corr_map:
    if i in imp.field.values:
        map_flds = [j for j in high_corr_map[i] if j in imp.field.values]
        if len(map_flds) > 0:
            print(i, map_flds)

real_word_per_iqr_standardized_normalized ['non_real_word_per_iqr_standardized_normalized']
non_real_word_per_iqr_standardized_normalized ['real_word_per_iqr_standardized_normalized']
outstanding_share_per_iqr_standardized_normalized ['offering_share_per_iqr_standardized_normalized', 'C5_prime_iqr_standardized_normalized']
offering_share_per_iqr_standardized_normalized ['outstanding_share_per_iqr_standardized_normalized', 'C5_prime_iqr_standardized_normalized']
C5_prime_iqr_standardized_normalized ['outstanding_share_per_iqr_standardized_normalized', 'offering_share_per_iqr_standardized_normalized']


In [68]:
predictors_y2 = feat_mat[feat_rfe[:10]]

# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []


#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,10):
    #Model building
    clf1 = LogisticRegression()

    
    # Splitting data into testing and training
    X2_train, X2_test, y2_train, y2_test = train_test_split(predictors_y2, y2, test_size=0.2, random_state=123)
    
    # Begin oversampling
    oversample = pd.concat([X2_train,y2_train],axis=1)
    max_size = oversample['Y2'].value_counts().max()
    lst = [oversample]
    for class_index, group in oversample.groupby('Y2'):
        lst.append(group.sample(max_size-len(group), replace=True))
    X2_train = pd.concat(lst)
    y2_train=pd.DataFrame.copy(X2_train['Y2'])
    del X2_train['Y2']
    
    # fitting model on oversampled data
    clf1.fit(X2_train, y2_train)
    
    y2_pred = clf1.predict(X2_test)
    
    
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=123)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(clf1, X2_train, y2_train, cv=kfold, scoring=scoring)
    
    #calculate f1-score and AUC
    
    clf1_roc_auc = roc_auc_score(y2_test, y2_pred)
    
    
    #calculate average f1-score and AUC
    f1_score_lst.append(precision_recall_fscore_support(y2_test, y2_pred, average='weighted')[2])
    auc_lst.append(clf1_roc_auc)
    
    
print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

confusion_matrix_y2 = confusion_matrix(y2_test, y2_pred)


print('Accuracy of classifier on test set: {:.3f}'.format(clf1.score(X2_test, y2_test)))

print("10-fold cross validation average accuracy of clf1: %.3f" % (results.mean()))

print('Confusion Matrix for Classfier:')
print(confusion_matrix_y2)

print('Classification Report for Classfier:')
print(classification_report(y2_test, y2_pred))

imp = pd.DataFrame(zip(predictors_y2.columns, clf1.coef_[0]))



/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is Fal

F1 0.6674; AUC 0.5982 
Accuracy of classifier on test set: 0.659
10-fold cross validation average accuracy of clf1: 0.630
Confusion Matrix for Classfier:
[[15 18]
 [27 72]]
Classification Report for Classfier:
              precision    recall  f1-score   support

           0       0.36      0.45      0.40        33
           1       0.80      0.73      0.76        99

    accuracy                           0.66       132
   macro avg       0.58      0.59      0.58       132
weighted avg       0.69      0.66      0.67       132



In [85]:
predictors_y2 = feat_mat[[i for i in feat_rfe[:13] if i not in ['non_real_word_per_iqr_standardized_normalized',
                                                              'outstanding_share_per_iqr_standardized_normalized',
                                                               'uncertain_words_percent_iqr_standardized_normalized']]]

# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []


#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,10):
    #Model building
    clf1 = LogisticRegression()

    
    # Splitting data into testing and training
    X2_train, X2_test, y2_train, y2_test = train_test_split(predictors_y2, y2, test_size=0.2, random_state=123)
    
    # Begin oversampling
    oversample = pd.concat([X2_train,y2_train],axis=1)
    max_size = oversample['Y2'].value_counts().max()
    lst = [oversample]
    for class_index, group in oversample.groupby('Y2'):
        lst.append(group.sample(max_size-len(group), replace=True))
    X2_train = pd.concat(lst)
    y2_train=pd.DataFrame.copy(X2_train['Y2'])
    del X2_train['Y2']
    
    # fitting model on oversampled data
    clf1.fit(X2_train, y2_train)
    
    y2_pred = clf1.predict(X2_test)
    
    
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=123)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(clf1, X2_train, y2_train, cv=kfold, scoring=scoring)
    
    #calculate f1-score and AUC
    
    clf1_roc_auc = roc_auc_score(y2_test, y2_pred)
    
    
    #calculate average f1-score and AUC
    f1_score_lst.append(precision_recall_fscore_support(y2_test, y2_pred, average='weighted')[2])
    auc_lst.append(clf1_roc_auc)
    
    
print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

confusion_matrix_y2 = confusion_matrix(y2_test, y2_pred)


print('Accuracy of classifier on test set: {:.3f}'.format(clf1.score(X2_test, y2_test)))

print("10-fold cross validation average accuracy of clf1: %.3f" % (results.mean()))

print('Confusion Matrix for Classfier:')
print(confusion_matrix_y2)

print('Classification Report for Classfier:')
print(classification_report(y2_test, y2_pred))

imp = pd.DataFrame(zip(predictors_y2.columns, clf1.coef_[0]))



/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is Fal

F1 0.6624; AUC 0.5982 
Accuracy of classifier on test set: 0.652
10-fold cross validation average accuracy of clf1: 0.604
Confusion Matrix for Classfier:
[[18 15]
 [31 68]]
Classification Report for Classfier:
              precision    recall  f1-score   support

           0       0.37      0.55      0.44        33
           1       0.82      0.69      0.75        99

    accuracy                           0.65       132
   macro avg       0.59      0.62      0.59       132
weighted avg       0.71      0.65      0.67       132



In [86]:
imp.columns = ['field', 'imp']
imp.sort_values(by = 'imp')
imp.sort_values('imp')

,field,imp
7,neg_words_percent_iqr_standardized_normalized,-0.123334
5,real_word_per_iqr_standardized_normalized,-0.068556
9,short_sentences_per_iqr_standardized_normalized,-0.056965
6,short_sentences_iqr_standardized_normalized,-0.025803
4,pos_words_percent_iqr_standardized_normalized,-0.005090
8,sales_per_share_iqr_standardized_normalized,-0.004233
0,C4_iqr_standardized_normalized,0.216916
3,offering_share_per_iqr_standardized_normalized,0.322708
1,C5_prime_iqr_standardized_normalized,0.491191
2,C6_prime_iqr_standardized_normalized,0.555448


In [87]:
for i in high_corr_map:
    if i in imp.field.values:
        map_flds = [j for j in high_corr_map[i] if j in imp.field.values]
        if len(map_flds) > 0:
            print(i, map_flds)

offering_share_per_iqr_standardized_normalized ['C5_prime_iqr_standardized_normalized']
C5_prime_iqr_standardized_normalized ['offering_share_per_iqr_standardized_normalized']


### PCA

In [88]:
pca = PCA(0.95)
# pca_transformed = pca.fit_transform(feat_mat[[i for i in feat_rfe[:10]]])

pca_transformed = pca.fit_transform(feat_mat[[i for i in feat_rfe[:13] if i not in ['non_real_word_per_iqr_standardized_normalized',
                                                              'outstanding_share_per_iqr_standardized_normalized',
                                                               'uncertain_words_percent_iqr_standardized_normalized']]])

In [91]:
predictors_y2 = pca_transformed

# lists for f1-score and AUC
f1_score_lst = []
auc_lst = []


#loop to calculate f1 and auc scores and present averages after 10 runs
for count in range (1,10):
    #Model building
    clf1 = LogisticRegression()

    
    # Splitting data into testing and training
    X2_train, X2_test, y2_train, y2_test = train_test_split(predictors_y2, y2, test_size=0.2, random_state=123)
    X2_train = pd.DataFrame(X2_train)
    X2_train['Y2'] = y2_train.values
    # Begin oversampling
    oversample = X2_train
    max_size = oversample['Y2'].value_counts().max()
    lst = [oversample]
    for class_index, group in oversample.groupby('Y2'):
        lst.append(group.sample(max_size-len(group), replace=True))
    X2_train = pd.concat(lst)
    y2_train=pd.DataFrame.copy(X2_train['Y2'])
    del X2_train['Y2']
    
    # fitting model on oversampled data
    clf1.fit(pd.DataFrame(X2_train), y2_train)
    
    y2_pred = clf1.predict(X2_test)
    
    
    #10-fold cross validation
    kfold = model_selection.KFold(n_splits=10, random_state=123)
    scoring = 'accuracy'
    results = model_selection.cross_val_score(clf1, X2_train, y2_train, cv=kfold, scoring=scoring)
    
    #calculate f1-score and AUC
    
    clf1_roc_auc = roc_auc_score(y2_test, y2_pred)
    
    
    #calculate average f1-score and AUC
    f1_score_lst.append(precision_recall_fscore_support(y2_test, y2_pred, average='weighted')[2])
    auc_lst.append(clf1_roc_auc)
    
    
print('F1 {:.4f}; AUC {:.4f} '.format(np.mean(f1_score_lst),np.mean(auc_lst)))

confusion_matrix_y2 = confusion_matrix(y2_test, y2_pred)


print('Accuracy of classifier on test set: {:.3f}'.format(clf1.score(X2_test, y2_test)))

print("10-fold cross validation average accuracy of clf1: %.3f" % (results.mean()))

print('Confusion Matrix for Classfier:')
print(confusion_matrix_y2)

print('Classification Report for Classfier:')
print(classification_report(y2_test, y2_pred))


/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is Fal

F1 0.6546; AUC 0.5881 
Accuracy of classifier on test set: 0.636
10-fold cross validation average accuracy of clf1: 0.598
Confusion Matrix for Classfier:
[[15 18]
 [30 69]]
Classification Report for Classfier:
              precision    recall  f1-score   support

           0       0.33      0.45      0.38        33
           1       0.79      0.70      0.74        99

    accuracy                           0.64       132
   macro avg       0.56      0.58      0.56       132
weighted avg       0.68      0.64      0.65       132



/opt/tljh/user/lib/python3.6/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


No immediate clear impact here